In [ ]:
import synapseclient
import json

import pandas as pd
import great_expectations as gx

from agoradatatools.gx import GreatExpectationsRunner

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations')

# Create Expectation Suite for Overall Scores Data

## Get Example Data File

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
overall_scores_data_file = syn.get("syn25740976").path


## Create Validator Object on Data File

In [ ]:
df = pd.read_json(overall_scores_data_file)
nested_columns = []
df = GreatExpectationsRunner.convert_nested_columns_to_json(df, nested_columns)
validator = context.sources.pandas_default.read_dataframe(df)
validator.expectation_suite_name = "overall_scores"

## Add Expectations to Validator Object For Each Column

In [ ]:
# ensembl_gene_id
validator.expect_column_values_to_be_of_type("ensembl_gene_id", "str")
validator.expect_column_values_to_not_be_null("ensembl_gene_id")
validator.expect_column_value_lengths_to_equal("ensembl_gene_id", 15)
# checks format and allowed chatacters
validator.expect_column_values_to_match_regex("ensembl_gene_id", "^ENSG\d{11}$")
validator.expect_column_values_to_be_unique("ensembl_gene_id")


In [ ]:
# hgnc_symbol
validator.expect_column_values_to_be_of_type("hgnc_symbol", "str")
validator.expect_column_value_lengths_to_be_between("hgnc_symbol", 1, 25)
validator.expect_column_values_to_match_regex("hgnc_symbol", "^[a-zA-Z0-9_.-]*$")

In [ ]:
# target_risk_score
validator.expect_column_values_to_be_of_type("target_risk_score", "float")
validator.expect_column_values_to_be_between("target_risk_score", 0, 5)
validator.expect_column_values_to_not_be_null("target_risk_score")

In [ ]:
# genetics_score
validator.expect_column_values_to_be_of_type("genetics_score", "float")
validator.expect_column_values_to_be_between("genetics_score", 0, 3)

In [ ]:
#  multi_omics_score
validator.expect_column_values_to_be_of_type("multi_omics_score", "float")
validator.expect_column_values_to_be_between("multi_omics_score", 0, 2)

## Save Expectation Suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
